<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [ ]:
from planet4 import io, markings, plotting

In [ ]:
%matplotlib inline

In [ ]:
image_id = 'APF0000gpu'
datapath = 'gold_comparison'
# datapath = "gold_per_obsid"
datapath = 'catalog_1.0b2'

In [ ]:
plotting.plot_finals(image_id, datapath=datapath)

In [ ]:
datapath = "gold_per_imageid"
datapath = "gold_per_obsid"

In [ ]:
from planet4.catalog_production import do_cluster_ids
from planet4 import projection
from planet4.projection import img_x_size, img_y_size
import seaborn as sns

class FNOTCH_REVIEWER:
    def __init__(self, image_id, datapath, via_obsid, plot_folder=''):
        self.image_id = io.check_and_pad_id(image_id)
        self.datapath = datapath
        self.via_obsid = via_obsid
        self.p4id = markings.ImageID(image_id)
        self.image_name = self.p4id.image_name
        self.plot_folder = plot_folder
        if via_obsid:
            self.pm = io.PathManager(obsid=self.p4id.image_name,
                                     datapath=datapath)
        else:
            self.pm = io.PathManager(image_id, datapath=datapath)
        self.db = io.DBManager()
        self.image_ids = []
        self.obsid_data = self.db.get_image_name_markings(self.p4id.image_name)
        self.check_data_exists()
        
    def check_data_exists(self):
        if self.via_obsid is True:
            # All should be there in via_obsid productions.
            return
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                image_id = self.get_other_image_id(dx, dy)
                if image_id is not None:
                    if not self.pm.clustering_logfile.exists():
                        print("clustering", image_id)
                        print(self.pm.fanfile)
                        print(self.pm.blotchfile)
                        do_cluster_ids(image_id, savedir=self.datapath,
                                       do_obsid=self.via_obsid)
                    
    @property
    def tile_coords(self):
        return self.p4id.tile_coords
    
    def get_other_image_id(self, dx, dy):
        coords = self.p4id.tile_coords
        data = self.obsid_data.query(
            "x_tile==@coords[0]+@dx and y_tile==@coords[1]+@dy")
        try:
            return data.image_id.iloc[0]
        except IndexError:
            return None
                
    @property
    def savename(self):
        d = Path('plots') / self.plot_folder
        Path(d).mkdir(exist_ok=True)
        return f"{d}/{self.image_id}_tiles_separate.png"
    
    def plot(self):
        fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(10,8),
                                 sharex=True, sharey=True)
        self.axes = axes
        plotting.plot_finals(self.image_id, datapath=self.datapath,
                             ax=axes[1][1],
                             via_obsid=self.via_obsid)
        axes[1][1].set_title(self.image_id)
        x = 1
        y = 1
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                image_id = self.get_other_image_id(dx, dy)
                if image_id is not None:
                    ax = axes[x+dy][y+dx]
                    plotting.plot_finals(image_id, datapath=self.datapath,
                                         ax=ax, via_obsid=self.via_obsid)
                    ax.set_title(f"{image_id}, {self.tile_coords[0]+dx}, {self.tile_coords[1]+dy}")
        fig.tight_layout()
        fig.subplots_adjust(top=0.91)
        t = "Fnotching "
        t = t+"via obsid" if self.via_obsid else t+"via image_id"
        fig.suptitle(t, fontsize=18)
        fig.savefig(self.savename, dpi=200)
        
    def combine_tiles(self):
        coords = self.p4id.tile_coords
        rows = []
        for dx in [-1, 0, 1]:
            cols = []
            for dy in [-1, 0, 1]:
                image_id = self.get_other_image_id(dx, dy)
                if image_id is not None:
                    self.image_ids.append(image_id)
                    cols.append(markings.ImageID(image_id).subframe)
            if len(cols) < 2:
                continue
            combo = np.vstack([cols[0], cols[1][100:]])
            if len(cols) > 2:
                combo = np.vstack([combo, cols[2][100:]])
            rows.append(combo)
        all_ = np.hstack([rows[0], rows[1][:, 100:]])
        if len(rows) > 2:
            all_ = np.hstack([all_, rows[2][:, 100:]])
        return all_
    
    @property
    def extent(self):
        p4id = markings.ImageID(self.image_ids[0])
        UL = projection.xy_to_hirise(0, 0, *p4id.tile_coords)
        p4id = markings.ImageID(self.image_ids[-1])
        LR = projection.xy_to_hirise(img_x_size, img_y_size, *p4id.tile_coords)
        return [UL[0], LR[0], LR[1], UL[1]]
    
    def plot_all_image(self, all_, ax=None):
        if ax is None:
            _, ax = plt.subplots(figsize=(len(self.image_ids), 9))
        ax.imshow(all_, origin='upper', extent=self.extent, aspect='equal')
        return ax
    
    def plot_all_in_one(self):
        all_ = self.combine_tiles()
        
        ax = self.plot_all_image(all_)
        
        palette = sns.color_palette("bright", 10)
        for id_, c in zip(self.image_ids, palette):
            plotting.plot_finals(id_, self.datapath,
                                 ax, scope='hirise', via_obsid=True,
                                 user_color=c)
        extent = self.extent
        ax.set_xlim(*extent[:2])
        ax.set_ylim(*extent[2:])
        for x in range(len(self.image_ids)//3 - 1):
            offset = (x + 1) * (markings.img_x_size - 100)
            ax.axvline(extent[0]+offset, linestyle='dashed', lw=1)
            ax.axvline(extent[0]+offset+100, linestyle='solid', lw=1)
            ax.axvline(extent[0]+offset+200, linestyle='dashed', lw=1)
        for y in range(3):
            offset = (y + 1) * (markings.img_y_size - 100)
            ax.axhline(extent[3]+offset, linestyle='dashed', lw=1)
            ax.axhline(extent[3]+offset+100, linestyle='solid', lw=1)
            ax.axhline(extent[3]+offset+200, linestyle='dashed', lw=1)
        t = f"{len(self.image_ids)} merged tiles around {self.image_id}"
        ax.set_title(t)
        savename = Path('plots') / self.plot_folder / f"{self.image_id}_overlap_merged.png"
        savename.parent.mkdir(exist_ok=True)
        plt.gcf().savefig(str(savename), dpi=200)
        print(f"Saved {savename}")

In [ ]:
datapath

In [ ]:
frev = FNOTCH_REVIEWER('gpu', datapath, via_obsid=True, plot_folder='overlap_merged')

In [ ]:
frev.plot_all_in_one()

In [ ]:
frev.plot()

In [ ]:
plotting.plot_four_tiles_finals(frev.p4id.image_name, datapath, 1, 13)
d = 'plots/gold_member_comparisons'
plt.gca().set_title('c10, 1, 13')
plt.savefig(f"{d}/fourtiles_c10.png", dpi=200)

In [ ]:
obsid_data = db.get_image_name_markings('ESP_020930_0980')

In [ ]:
obsid_data = db.get_image_name_markings('ESP_012076_0945')

In [ ]:
image_ids = ["APF0000b20",
"APF0000b2s",
"APF0000b1r",
"APF0000bm0",
"APF0000bln",
"APF0000bj5",
"APF0000bgm",
"APF0000bdl",
"APF0000ps6",
"APF0000ps1",
"APF0000pu3",
"APF0000psx",
"APF0000ps2",]

In [ ]:
p4id = markings.ImageID(image_ids[0])
frev = FNOTCH_REVIEWER(p4id.imgid, p4id.image_name, via_obsid=True)

In [ ]:
frev.plot_all_in_one()

In [ ]:
frev.plot()

In [ ]:
frev.image_name

In [ ]:
for id_ in image_ids:
    p4id = markings.ImageID(id_)
    frev = FNOTCH_REVIEWER(p4id.imgid, p4id.image_name, via_obsid=True,
                           plot_folder='check_overlap')
    print(id_)
    frev.plot()
    frev.plot_all_in_one()
    plt.close('all')

In [ ]:
frev.plot()

In [ ]:
xmax = obsid_data.x_tile.max()
ymax = obsid_data.y_tile.max()

In [ ]:
def get_tile_image(df, xtile, ytile):
    filtered = df.query('x_tile=={} and y_tile=={}'.format(xtile, ytile))
    return io.get_subframe(filtered.image_url.iloc[0])

def get_four_tiles_df(df, x0, y0):
    query = ('x_tile > {} and x_tile < {} and y_tile > {} and y_tile < {}'.
             format(x0-1, x0+2, y0-1, y0+2))
    return df.query(query).sort_values(by=['x_tile', 'y_tile'])
         
def get_four_tiles_img(df, x0, y0):
    tiles = []
    # loop along columns (= to the right)
    for xtile in [x0, x0+1]:
        # loop along rows (= down)
        for ytile in [y0, y0+1]:
            tiles.append(get_tile_image(df, xtile, ytile))
    
    # tiles[0] and tiles[1] are the left most tiles
    # we have overlap of 100 pixels in all directions
    left = np.vstack([tiles[0], tiles[1][100:]])
    right = np.vstack([tiles[2], tiles[3][100:]])
    # now slicing on axis=1, because I combine in column-direction
    all_ = np.hstack([left, right[:, 100:]])
    return all_

def browse_images(df):
    xmax = df.x_tile.max()
    ymax = df.y_tile.max()
    def view_image(xtile=1, ytile=1):
        img = get_four_tiles_img(df, xtile, ytile)
        print(img.shape)
        plt.imshow(img, origin='upper', aspect='auto')
        plt.title(f'x_tile: {xtile}, y_tile: {ytile}')
        plt.show()
    interact(view_image, xtile=(1, xmax-1), ytile=(1, ymax-1))

In [ ]:
l_s = [180.3, 189.7, 195.1, 199.4, 209.2, 243.8]

In [ ]:
meta_data = [123, 131, 142, 133, 156, 158]
spice = [120.4, 121.5, 130.3, 122.2, 123.8, 122.7 ]

In [ ]:
plt.plot(l_s, meta_data, '-*', label='meta_data and website')
plt.plot(l_s, spice, '-*', label='spice')
plt.title("Unprojected North Azimuth discrepancies")
plt.xlabel("Solar longitude")
plt.ylabel("Unprojected North Azimuth")
plt.legend()
plt.savefig("north_azimuth_deltas.png", dpi=200)